In [2]:
#!/usr/bin/env python3
import sys
srcdir = "/project/mchaisso_100/cmb-16/tsungyul/work/vntr/danbing-tk/script/"
sys.path.insert(0, srcdir)

In [3]:
import numpy as np
import pandas as pd
import vntrutils as vu
import utils
# import scipy.stats as stats
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
# %load_ext autoreload
# %autoreload 2

In [4]:
from collections import defaultdict
import pickle
# import networkx as nx
# import pygraphviz
# import collections
import gc
# import glob
# import warnings
import statsmodels.api as sm
# from statsmodels.stats.multitest import fdrcorrection as fdr
# from scipy import stats
import gzip
# from sklearn.decomposition import PCA
# from scipy.stats import chi2
# import re
# import json

In [5]:
matplotlib.rc('font', size=7)
matplotlib.rc('axes', titlesize=7)
matplotlib.rc('xtick', labelsize=5)
matplotlib.rc('ytick', labelsize=5)
%load_ext autoreload
%autoreload 2

In [10]:
def test():
    with open("/project/mchaisso_100/cmb-17/vntr_genotyping/rpgg2_k21_84k/hgsvc/clinic/cdbg/v1/ki_tr.ccki_tr.pickle", 'rb') as f:
        ki_tr, ccki_tr = pickle.load(f)
    return ki_tr, ccki_tr
ki_tr, ccki_tr = test()

In [11]:
def get_ki(tri):
    si = ki_tr[tri-1] if tri else 0
    ei = ki_tr[tri]
    return si, ei
def get_ccki(tri):
    si = ccki_tr[tri-1] if tri else 0
    ei = ccki_tr[tri]
    return si, ei

# Bias correction for GTEx genotypes

In [7]:
def test():
    with open("/project/mchaisso_100/cmb-17/vntr_genotyping/rpgg2_k21_84k/hgsvc/clinic/bias/invariant_kmer/ikmer.ikstats.pickle", 'rb') as f:
        ikmer, ikstats = pickle.load(f)
    return ikmer, ikstats
ikmer, ikstats = test()

In [8]:
cov = np.loadtxt("/project/mchaisso_100/cmb-17/vntr_genotyping/analysis/read_depth/ctrl/gtex.cov.tsv", usecols=1)

In [ ]:
def test():
    with open(f"/scratch1/tsungyul/tmp/gt.1.pickle", 'rb') as f:
        gt = pickle.load(f)
    with open(f"/project/mchaisso_100/cmb-17/vntr_genotyping/rpgg2_k21_84k/hgsvc/gtex/gt_pickle/repickle.python3.9/gt.1.pickle", 'wb') as f:
        pickle.dump(gt, f, protocol=pickle.HIGHEST_PROTOCOL)
test()

In [9]:
def test():
    ng = np.loadtxt("/project/mchaisso_100/cmb-16/tsungyul/work/vntr/hapdb/config/genomes.gtex.txt", dtype=object).size
    with open("/project/mchaisso_100/cmb-17/vntr_genotyping/rpgg2_k21_84k/hgsvc/clinic/cdbg/v1/tr_cck_ns.pickle", 'rb') as f:
        tr_cck_ns = pickle.load(f)
    nk = ki_tr[-1]
    ntr = len(ki_tr)
    
    NB = 10
    BS = ng // NB
    bias = np.full([ng, ntr], np.nan)
    for i in range(NB):
        BS_ = BS if i != NB-1 else ng - BS*i
        i0s = i * BS
        i0e = i0s + BS_
        print(f"batch {i}: loading gt", end="")
        with open(f"/project/mchaisso_100/cmb-17/vntr_genotyping/rpgg2_k21_84k/hgsvc/gtex/gt_pickle/gt.{i}.pickle", 'rb') as f:
            agt = pickle.load(f).astype(float) / cov[i0s:i0e,None]
        for tri in range(ntr):
            if tri % 1000 == 0:
                print(".", end="")
            si, ei = get_ki(tri)
            inv = np.nonzero(ikmer[si:ei])[0] + si
            if inv.size:
                bias[i0s:i0e, tri] = np.mean(agt[:,inv] / ikstats[0,inv], axis=1)
        print()
        # del agt
        # gc.collect()
    bias[bias==0] = np.nan
    bias /= np.nanmean(bias, axis=0)
    n0 = np.sum(~np.isfinite(bias))
    print(f"{n0} cells of nan's")
    return bias

bias = test()

batch 0: loading gt.................................................................................
batch 1: loading gt.................................................................................
batch 2: loading gt.................................................................................
batch 3: loading gt.................................................................................
batch 4: loading gt.................................................................................
batch 5: loading gt.................................................................................
batch 6: loading gt.................................................................................
batch 7: loading gt.................................................................................
batch 8: loading gt.................................................................................
batch 9: loading gt........................................................................

/tmp/SLURM_9845618/ipykernel_7119/3750106175.py:29: RuntimeWarning: Mean of empty slice
  bias /= np.nanmean(bias, axis=0)


3520971 cells of nan's


In [10]:
pd.DataFrame(bias)

,0,1,2,3,4,5,6,7,8,9,...,80508,80509,80510,80511,80512,80513,80514,80515,80516,80517
0,1.445350,1.311181,1.466103,0.999367,1.719675,NaN,1.225360,1.081811,NaN,1.315846,...,1.751865,2.014984,1.294471,1.347526,1.213396,1.635720,1.493033,2.432265,1.403226,1.509533
1,1.394555,0.938126,1.389723,1.186259,1.576146,NaN,1.473246,1.016119,NaN,1.258831,...,1.155213,1.203710,0.865699,0.945337,0.811516,0.900387,1.030346,0.858214,0.684619,0.673129
2,0.948807,0.826499,1.178828,0.940856,1.117730,NaN,1.121415,1.119948,NaN,1.376825,...,0.849115,0.746642,0.698708,0.875612,0.650822,0.806310,0.951127,0.759671,0.614613,0.841087
3,1.146230,1.425681,1.395216,1.000258,0.973174,NaN,0.997782,1.883899,NaN,1.654735,...,1.068061,1.109623,0.999791,1.166510,0.951214,0.696997,0.775892,1.073341,0.798879,1.327245
4,1.203270,1.244994,1.428072,1.339697,1.358357,NaN,1.410148,1.457863,NaN,1.197611,...,0.671996,1.146380,0.557170,0.905058,0.520475,0.753244,0.898637,0.871257,0.533988,0.872749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,0.916213,1.221898,1.236653,1.002084,1.152078,NaN,1.276948,1.126848,NaN,1.025336,...,1.721028,1.651361,2.443208,1.500924,1.327826,1.616015,2.297157,2.130944,1.259901,2.100690
875,1.426672,0.680185,1.711932,1.729307,1.759104,NaN,1.597813,1.077619,NaN,1.566764,...,1.949701,2.351928,2.368467,1.838444,1.244059,1.904524,2.191950,2.420843,2.610124,2.603747
876,0.961147,1.160276,1.413451,1.205609,1.409398,NaN,1.564082,1.337167,NaN,1.267149,...,0.496496,0.981287,0.888358,0.944783,1.378401,1.035549,0.704856,0.951897,1.116598,1.005512
877,0.930173,0.830056,1.111363,1.294423,1.419679,NaN,1.053540,0.888242,NaN,0.851225,...,0.819429,0.986135,0.558425,0.832650,0.855398,0.685338,0.978693,0.600650,0.841698,1.023044


In [11]:
def test():
    with open("/project/mchaisso_100/cmb-17/vntr_genotyping/rpgg2_k21_84k/hgsvc/gtex/bias/bias.v1.pickle", 'wb') as f:
        pickle.dump(bias, f)
# test()

In [6]:
bias = pickle.load(open("/project/mchaisso_100/cmb-17/vntr_genotyping/rpgg2_k21_84k/hgsvc/gtex/bias/bias.v1.pickle", 'rb'))

In [16]:
def test():
    gloci = np.all(pd.read_csv("/project/mchaisso_100/cmb-17/vntr_genotyping/rpgg2_k21_84k/hgsvc/clinic/analysis/80518_loci.alu.mismap_1.r2.v2.filter.tsv", sep="\t", index_col=0).iloc[:,3:].values, axis=1)
    droppedloci = ~np.isfinite(bias[0])
    n0 = np.sum(droppedloci)
    n1 = np.sum(gloci & droppedloci)
    print(f"{n0} loci have nan bias")
    print(f"{n1}/{n0} overlapped with loci passing QC")
test()

4330 loci have nan bias
3434/4330 overlapped with loci passing QC


In [20]:
def test():
    with open("/project/mchaisso_100/cmb-17/vntr_genotyping/rpgg2_k21_84k/hgsvc/gtex/gt_pickle/acgt.pickle", 'rb') as f:
        return pickle.load(f).astype(float) # ng x ncck
acgt = test()

In [21]:
def test():
    ntr = len(ccki_tr)
    acgt_bc = np.copy(acgt)
    for tri in range(ntr):
        si, ei = get_ccki(tri)
        acgt_bc[:,si:ei] /= (bias[:,tri][:,None])
    return acgt_bc
acgt_bc = test()

In [22]:
pd.DataFrame(acgt_bc)

,0,1,2,3,4,5,6,7,8,9,...,4456871,4456872,4456873,4456874,4456875,4456876,4456877,4456878,4456879,4456880
0,1.894146,1.255609,3.270057,2.885868,3.302633,0.0,0.587190,0.183363,0.927947,0.000000,...,0.001674,0.281235,0.234362,0.367481,1.044098,0.478532,0.774123,0.237362,0.528670,0.539459
1,2.073185,0.960212,3.207981,2.918540,3.157987,0.0,1.362505,0.437948,1.059949,0.000000,...,0.000000,0.577133,0.037234,0.734962,2.077995,0.941070,1.349007,0.474724,1.091864,1.210546
2,2.167117,1.044911,3.183258,2.931552,3.098819,0.0,1.028945,0.473243,1.252413,0.000000,...,0.000000,0.600687,0.375429,0.734962,1.692359,0.955431,1.311142,0.474724,0.847721,0.881630
3,2.003076,1.118083,3.313863,2.862812,3.281501,0.0,0.658137,0.365783,0.745243,0.000000,...,0.000000,0.426290,0.465890,0.734962,1.245379,0.795195,1.273465,0.474724,0.696261,0.542541
4,2.181890,0.912341,3.130889,2.959115,3.311323,0.0,0.637477,0.290904,0.817979,0.000000,...,0.000000,0.594421,0.384958,0.734962,1.077376,0.472729,0.946622,0.474724,0.583232,0.576450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,1.727134,1.340911,2.956358,3.050973,3.198306,0.0,0.519366,0.457593,0.921455,0.002025,...,0.000000,0.266072,0.007548,0.367481,0.914992,0.498595,0.764407,0.237362,0.483912,0.436440
875,2.117852,0.846391,2.945740,3.056561,3.217353,0.0,1.331643,0.453655,2.280460,0.000000,...,0.000000,0.268834,0.278504,0.367481,0.774532,0.407027,0.622775,0.237362,0.384095,0.406392
876,1.920030,1.121865,3.079142,2.986350,3.453877,0.0,0.504903,0.347733,0.459803,0.000000,...,0.007835,0.650333,0.407438,0.734962,2.628513,1.425916,1.876642,0.474724,1.180875,1.260985
877,1.863256,1.098356,2.785670,3.140809,3.048964,0.0,0.838534,0.119642,1.344305,0.000000,...,0.000000,0.713437,0.000000,0.734962,2.225109,0.982300,1.480021,0.474724,1.072317,1.144922


In [23]:
def test():
    with open("/project/mchaisso_100/cmb-17/vntr_genotyping/rpgg2_k21_84k/hgsvc/gtex/gt_pickle/acgt_bc.v1.pickle", 'wb') as f:
        pickle.dump(acgt_bc, f, protocol=pickle.HIGHEST_PROTOCOL)
# test()

In [9]:
def test():
    with open("/project/mchaisso_100/cmb-17/vntr_genotyping/rpgg2_k21_84k/hgsvc/gtex/gt_pickle/acgt_bc.v1.pickle", 'rb') as f:
        return pickle.load(f)
acgt_bc = test()

In [ ]:
def test():
    ntr = len(ki_tr)
    for tri in range(ntr):
        si, ei = get_ccki(tri)
        xs = np.sum(acgt_bc[:,si:ei], axis=1)
        utils.hist(xs, "Read TR dosage", "Count", f"{tri}", bins=50)
        input()
test()

In [18]:
def test():
    with open("/project/mchaisso_100/cmb-17/vntr_genotyping/rpgg2_k21_84k/hgsvc/gtex/genotype/gtex.80518x879.akms.pickle", 'rb') as f:
        return pickle.load(f)
akms = test()

In [25]:
pd.DataFrame(akms)

,0,1,2,3,4,5,6,7,8,9,...,869,870,871,872,873,874,875,876,877,878
0,274.624940,259.347199,179.392492,216.835655,227.340417,206.786075,185.550319,157.939153,194.511432,182.887262,...,174.183396,168.412230,211.880699,176.028533,206.772705,171.462219,264.179840,182.045607,170.467737,225.264635
1,422.996023,368.345881,315.491215,333.159418,351.871590,288.355775,276.868428,270.056452,395.633302,353.757231,...,372.444559,387.909299,308.521888,356.652555,242.808667,297.533595,348.306866,237.864573,316.338068,404.442990
2,4802.793135,4827.609432,4098.220049,5014.043235,5012.409322,4459.796549,3750.101498,2591.152662,3914.627678,4535.804878,...,4527.395834,3887.663045,4167.950565,4436.990298,4431.418094,4230.194900,5967.025879,4908.339793,3896.131971,4966.283077
3,248.826938,273.023962,223.969672,227.126872,282.592247,305.128832,251.045456,202.761699,272.446511,316.937661,...,199.663461,203.180419,250.945929,307.562994,108.321116,318.733145,356.097793,244.845673,266.571887,263.951596
4,475.601797,449.096296,315.491215,386.775761,367.437987,458.045237,238.292658,258.552242,344.934362,294.969930,...,338.968407,354.849021,356.054171,376.556196,381.103906,367.759115,496.858865,355.707927,334.936106,346.141453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80513,2640.874987,1456.351480,1307.654332,1136.324444,1223.796961,2960.781148,2243.828762,1118.467558,2716.606349,1099.065637,...,1101.818882,2188.535529,1099.140248,2519.910535,1331.655210,2603.845196,3107.493554,1663.530490,1105.197732,3247.886748
80514,108.273431,75.541742,70.017260,56.376670,65.846450,103.301214,84.970563,45.115026,116.179159,55.450804,...,45.947658,108.391355,58.335075,150.342488,47.672257,167.955768,160.425670,51.433403,71.535313,137.684756
80515,613.419152,211.861993,178.023519,270.962059,206.831837,473.655198,308.548455,160.912698,538.906745,217.758082,...,181.910957,349.145818,142.012914,464.631330,252.282195,516.638505,608.066853,251.886441,143.527721,520.376297
80516,134.071433,65.411991,58.723234,76.329031,51.019901,114.687304,170.662438,103.391649,166.651130,145.181897,...,95.266801,113.942066,75.064803,240.700150,74.691614,120.377415,249.384570,106.685528,80.420090,181.697951


In [24]:
def test():
    akms_bc = np.copy(akms)
    akms_bc /= bias.T
    return akms_bc
akms_bc = test()

In [26]:
pd.DataFrame(akms_bc)

,0,1,2,3,4,5,6,7,8,9,...,869,870,871,872,873,874,875,876,877,878
0,190.005882,185.971268,189.071667,189.172918,188.935467,188.693885,187.890092,181.855654,183.410710,197.297531,...,189.404077,184.626458,188.997382,190.189883,190.316355,187.142363,185.172063,189.404491,183.264504,184.990718
1,322.606951,392.639983,381.719929,233.684423,282.629192,394.021460,402.614711,332.810514,401.527955,211.407896,...,294.605532,303.974946,460.913080,276.574720,157.705304,243.501150,512.076635,205.006957,381.104459,237.724717
2,3275.889656,3473.792310,3476.520599,3593.738497,3509.912612,3451.342498,3440.220741,3522.190435,3337.611568,3546.336004,...,3790.461717,3520.199727,3503.122231,3524.808688,3448.516419,3420.681781,3485.551067,3472.591682,3505.723017,3571.234322
3,248.984631,230.155480,238.048788,227.068319,210.937507,294.405963,239.847601,242.790990,227.798704,221.057516,...,190.167151,252.869579,195.068549,289.947655,250.905447,318.070297,205.919421,203.088782,205.938719,227.207621
4,276.564960,284.933126,282.260718,397.437451,270.501844,272.934584,333.636751,267.206133,406.752115,333.189020,...,309.630384,317.899689,368.617510,188.981274,326.559172,319.213650,282.449891,252.382943,235.923785,334.128014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80513,1614.503394,1617.471934,1621.776762,1630.313899,1624.701529,1622.856641,1617.295123,1639.756415,1625.256451,1614.328442,...,1608.168700,1627.221967,1621.597226,1618.357257,1616.064579,1611.275803,1631.638075,1606.423895,1612.631497,1627.411732
80514,72.519138,73.316902,73.615027,72.660497,73.273668,72.873326,73.089389,73.031967,72.851205,72.558087,...,73.365146,72.651937,72.406809,72.954918,72.950800,73.114631,73.188568,72.970103,73.092703,72.632336
80515,252.200818,246.863856,234.343028,252.447436,237.394839,246.333033,251.771238,245.572108,257.741907,240.317653,...,250.225006,244.644040,254.974139,248.520299,262.551700,242.445869,251.179799,264.615137,238.954002,262.959837
80516,95.545120,95.545120,95.545120,95.545120,95.545120,95.545120,95.545120,95.545120,95.545120,95.545120,...,95.545120,95.545120,95.545120,95.545120,95.545120,95.545120,95.545120,95.545120,95.545120,95.545120


In [27]:
def test():
    with open("/project/mchaisso_100/cmb-17/vntr_genotyping/rpgg2_k21_84k/hgsvc/gtex/genotype/gtex.80518x879.akms_bc.v1.pickle", 'wb') as f:
        pickle.dump(akms_bc, f, protocol=pickle.HIGHEST_PROTOCOL)
test()